In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr
from src.models.classifiers import *
from src.trainer.trainer import TrainerClassifier, Model_class

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1024
INIT_PARAM = 512

In [3]:
df = pd.read_csv('../data/prepared_data.csv')

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42,
                                                 train_size=0.7)

In [5]:
df_train = pd.concat([X_train, y_train], axis = 1)
df_test = pd.concat([X_test, y_test], axis = 1)

In [6]:
train_dataset = ClassifierDataset(df_train)
val_dataset = ClassifierDataset(df_test)

In [7]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [23]:
model = BaselineClassifier(X_train.shape[1], INIT_PARAM)

In [24]:
for param in model.parameters():
  print(param.data)

tensor([[ 0.0820,  0.0891, -0.0251,  ..., -0.0522,  0.0630,  0.0946],
        [-0.0787,  0.0933,  0.0201,  ..., -0.0152,  0.0827,  0.0159],
        [-0.0501,  0.0274, -0.0494,  ..., -0.0847, -0.0495, -0.0303],
        ...,
        [ 0.0111,  0.0762, -0.0176,  ..., -0.0056, -0.0194,  0.0341],
        [ 0.0992,  0.0367, -0.0898,  ...,  0.0332,  0.0937,  0.0533],
        [-0.0021, -0.0082, -0.0503,  ...,  0.0172, -0.0787, -0.0293]])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [25]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

709842

In [26]:
# loss_func = FocalLoss(gamma=2)
loss_func = nn.CrossEntropyLoss()
loss = LossWrapper(loss_func)
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.90)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'BL_{INIT_PARAM}_{BATCH_SIZE}',
    project="Internship_project",
    dir = '../logs/'
)

In [27]:
additional_params = dict(loss = loss_func,
                        p = 0,
                        sampling = 'None',
                        batch_size = BATCH_SIZE,
                        init_parammeters = INIT_PARAM,
                        features_amount = df.shape[1])

In [28]:
trainer = TrainerClassifier(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  additional_params=additional_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/classifier/',
                  saving_model=False
                  )

In [29]:
trainer.train_model(learning_params)
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,█▇▇▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▇██████████████████████████████████████
test_auc_score,▃▄▁▂▄▄▂▇▂▄▄▅▅▆▆▆▇▇▇▆▇▇▇███▇█▇▇▇██▇▇▇█▇██
test_f1_score,▁▅▇▇█▇▇█▇▇█▇████████████████████████████
test_fpr,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,█▆▄▃▄▃▁▄▃▁▂▂▃▃▂▂▄▄▄▃▃▃▃▃▃▃▄▄▄▄▃▄▄▄▃▃▄▃▄▄
train_accuracy,▁▇██████████████████████████████████████
train_auc_score,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
train_f1_score,▁▅▅▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇█▇▇██▇███████████████
